# TigerGraph Data Science Library 101 - Topological Link Predication Algorithm
This notebook shows the examples of using the most common topological link predication algorithms in TigerGraph Graph Science Library. More detailed explanations of these algorithms can be found in the official documentation (https://docs.tigergraph.com/graph-ml/current/link-prediction/).


## Step1: Setting things up
- Connect and Load data
- Visualize the graph schema 
- Get basic stats, e.g., counts of nodes & edges

### Create connection

In [1]:
import json
import pandas as pd
from pyTigerGraph import TigerGraphConnection

# Read in DB configs
with open('../config.json', "r") as config_file:
    config = json.load(config_file)

conn = TigerGraphConnection(
    host=config["host"],
    username=config["username"],
    password=config["password"],
)

### Download movie dataset

In [2]:
from pyTigerGraph.datasets import Datasets

dataset_social = Datasets("social")

Downloading:   0%|          | 0/1970 [00:00<?, ?it/s]

### Ingest data

In [3]:
conn.ingestDataset(dataset_social, getToken=config["getToken"])

---- Checking database ----
A graph with name social already exists in the database. Skip ingestion.


### Visualize schema

In [4]:
from pyTigerGraph.visualization import drawSchema

drawSchema(conn.getSchema(force=True))

CytoscapeWidget(cytoscape_layout={'name': 'circle', 'animate': True, 'padding': 1}, cytoscape_style=[{'selecto…

### Print graph stats

In [5]:
vertices = conn.getVertexTypes()
total_count = 0
for vertex in vertices:
    vertex_cnt = conn.getVertexCount(vertex)
    total_count += vertex_cnt
    print("Node count: ({} : {}) ".format(vertex, vertex_cnt))
print("Total node count: ", total_count)

Node count: (Person : 12) 
Total node count:  12


In [6]:
import pprint
edge_count = conn.getEdgeCount()
print("Edges count: total ", sum(edge_count.values()))
pprint.pprint(edge_count) 

Edges count: total  39
{'Coworker': 11, 'Friend': 14, 'reverse_Friend': 14}


## Step 2: Leveraging pyTigerGraph’s featurizer to run Topological Link Prediction algorithms

pyTigerGraph provides a full suit of data science capabilities, and in this tutorial, we will showcase how to use featurizer to list out all available Topological Link Prediction algorithms in our GDS library, and to run a few popular algorithms as an example.

In [7]:
feat = conn.gds.featurizer()

In [8]:
feat.listAlgorithms("Topological Link Prediction")

Available algorithms for Topological Link Prediction:
  adamic_adar:
    01. name: tg_adamic_adar
  common_neighbors:
    02. name: tg_common_neighbors
  preferential_attachment:
    03. name: tg_preferential_attachment
  resource_allocation:
    04. name: tg_resource_allocation
  same_community:
    05. name: tg_same_community
  total_neighbors:
    06. name: tg_total_neighbors
Call runAlgorithm() with the algorithm name to execute it


## tg_adamic_adar

The Adamic/Adar index is a measure according to the number of shared links between two vertices. It is defined as the sum of the inverse logarithmic degree centrality of the neighbors shared by the two vertices. This algorithm ignores edge weights. (https://docs.tigergraph.com/graph-ml/current/link-prediction/adamic-adar)


## Input Parameters

* VERTEX v_source: The first vertex to compare {"id": "vertex_id", "type": "vertex_type"}
* VERTEX v_target: The second vertex to compare with the first {"id": "vertex_id", "type": "vertex_type"}
* SET<STRING> e_type_set: Edge types to traverse 
* BOOL print_results: if True, print result (True by default)

In [9]:
params = {
    "v_source": {"id": "Alex", "type": "Person"},
    "v_target": {"id": "Bob", "type": "Person"},
    "e_type_set": ["Coworker"],
    "print_results": True
}

In [10]:
import csv
import os
import time
import psutil
!pip install memory_profiler
%load_ext memory_profiler

algo_performance_out = '/home/tigergraph/GraphML/output/algorithm_' + config["job_id"] + '.csv'

start_time = time.time()

algo_memory = %memit -r 1 -o feat.runAlgorithm("tg_adamic_adar", params=params)

algo_memory = str(algo_memory)

start = algo_memory.find(": ") + 1
end = algo_memory.find("M")

algo_memory = algo_memory[start:end].strip()

execution_time = time.time() - start_time

cpu_usage = psutil.cpu_percent(4)

print('The CPU usage is: ', cpu_usage)

# print('RAM memory % used:', psutil.virtual_memory()[2])

host_memory = psutil.virtual_memory()[3]/1000000000

print('RAM Used (GB):', host_memory)

print ('tg_adamic_adar executed successfully')

print ('execution time: ' + str(execution_time) + ' seconds\n')

algo_id = "tg_adamic_adar_" + config["job_id"]

nb_id = "algos_topologicalLinkPrediction.ipynb_" + config["job_id"]

data = [algo_id, "false" ,cpu_usage, algo_memory, execution_time, host_memory, "3.8", "no error", nb_id]

with open(algo_performance_out, mode='a+', encoding='utf-8') as f:
    writer = csv.writer(f) 
    writer.writerow(data)

/opt/conda/lib/python3.9/site-packages/memory_profiler.py:1136: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ipython_version = LooseVersion(IPython.__version__)
/opt/conda/lib/python3.9/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


Installing and optimizing the queries, it might take a minute...
Queries installed successfully
peak memory: 124.61 MiB, increment: 2.23 MiB
The CPU usage is:  30.1
RAM Used (GB): 11.353780224
tg_adamic_adar executed successfully
execution time: 35.717833042144775 seconds



In [13]:
results = feat.runAlgorithm("tg_adamic_adar", params=params)

## Results

Returns Adamic Adar index between the two given vertices. If the two vertices do not have common neighbors, the algorithm will return a division by 0 error

In [14]:
df_adamic_adar = pd.json_normalize(results)

display(df_adamic_adar)

,@@sum_closeness
0,3.32193


## tg_common_neighbors

A vertex 𝐴 that is connected to vertices 𝐵 and 𝐶 is considered to be a "common neighbor" of 𝐵 and 𝐶. The common neighbors algorithm counts the number of common neighbors between two vertices. This algorithm ignores edge weights. (https://docs.tigergraph.com/graph-ml/current/link-prediction/common-neighbors)


## Input Parameters

* VERTEX v_source: The first vertex to compare {"id": "vertex_id", "type": "vertex_type"}
* VERTEX v_target: The second vertex to compare with the first {"id": "vertex_id", "type": "vertex_type"}
* SET<STRING> e_type_set: Edge types to traverse 
* BOOL print_results: if True, print result (True by default)

In [15]:
params = {
    "v_source": {"id": "Alex", "type": "Person"},
    "v_target": {"id": "Bob", "type": "Person"},
    "e_type_set": ["Coworker"],
    "print_results": True
}

In [16]:
start_time = time.time()

algo_memory = %memit -r 1 -o feat.runAlgorithm("tg_common_neighbors", params=params)

algo_memory = str(algo_memory)

start = algo_memory.find(": ") + 1
end = algo_memory.find("M")

algo_memory = algo_memory[start:end].strip()

execution_time = time.time() - start_time

cpu_usage = psutil.cpu_percent(4)

print('The CPU usage is: ', cpu_usage)

# print('RAM memory % used:', psutil.virtual_memory()[2])

host_memory = psutil.virtual_memory()[3]/1000000000

print('RAM Used (GB):', host_memory)

print ('tg_common_neighbors executed successfully')

print ('execution time: ' + str(execution_time) + ' seconds\n')

algo_id = "tg_common_neighbors_" + config["job_id"]

nb_id = "algos_topologicalLinkPrediction.ipynb_" + config["job_id"]

data = [algo_id, "false" ,cpu_usage, algo_memory, execution_time, host_memory, "3.8", "no error", nb_id]

with open(algo_performance_out, mode='a+', encoding='utf-8') as f:
    writer = csv.writer(f) 
    writer.writerow(data)

Installing and optimizing the queries, it might take a minute...
Queries installed successfully
peak memory: 126.42 MiB, increment: 0.11 MiB
The CPU usage is:  33.9
RAM Used (GB): 11.365081088
tg_common_neighbors executed successfully
execution time: 32.86519742012024 seconds



In [17]:
results = feat.runAlgorithm("tg_common_neighbors", params=params)

## Results

Returns the number of common neighbors between two vertices expressed as a closeness value.

In [18]:
df_common_neighbors = pd.json_normalize(results)

display(df_common_neighbors)

,closeness
0,1


## tg_preferential_attachment

Preferential Attachment is a measure to compute the closeness of vertices based on the number of their neighbors. The algorithm returns the product of the number of neighbors of the first vertex and the number of neighbors of the second vertex. (https://docs.tigergraph.com/graph-ml/current/link-prediction/preferential-attachment)

## Input Parameters

* VERTEX v_source: The first vertex to compare {"id": "vertex_id", "type": "vertex_type"}
* VERTEX v_target: The second vertex to compare with the first {"id": "vertex_id", "type": "vertex_type"}
* SET<STRING> e_type_set: Edge types to traverse
* BOOL print_results: if True, print result (True by default)

In [19]:
params = {
    "v_source": {"id": "Alex", "type": "Person"},
    "v_target": {"id": "Bob", "type": "Person"},
    "e_type_set": ["Coworker"],
    "print_results": True
}

In [20]:
start_time = time.time()

algo_memory = %memit -r 1 -o feat.runAlgorithm("tg_preferential_attachment", params=params)

algo_memory = str(algo_memory)

start = algo_memory.find(": ") + 1
end = algo_memory.find("M")

algo_memory = algo_memory[start:end].strip()

execution_time = time.time() - start_time

cpu_usage = psutil.cpu_percent(4)

print('The CPU usage is: ', cpu_usage)

# print('RAM memory % used:', psutil.virtual_memory()[2])

host_memory = psutil.virtual_memory()[3]/1000000000

print('RAM Used (GB):', host_memory)

print ('tg_preferential_attachment executed successfully')

print ('execution time: ' + str(execution_time) + ' seconds\n')

algo_id = "tg_preferential_attachment_" + config["job_id"]

nb_id = "algos_topologicalLinkPrediction.ipynb_" + config["job_id"]

data = [algo_id, "false" ,cpu_usage, algo_memory, execution_time, host_memory, "3.8", "no error", nb_id]

with open(algo_performance_out, mode='a+', encoding='utf-8') as f:
    writer = csv.writer(f) 
    writer.writerow(data)

Installing and optimizing the queries, it might take a minute...
Queries installed successfully
peak memory: 126.51 MiB, increment: 0.38 MiB
The CPU usage is:  23.9
RAM Used (GB): 11.388018688
tg_preferential_attachment executed successfully
execution time: 33.8450243473053 seconds



In [23]:
results = feat.runAlgorithm("tg_preferential_attachment", params=params)

## Results

The product of the number of neighbors of the two vertices. 

In [24]:
df_preferential_attachment = pd.json_normalize(results)

display(df_preferential_attachment)

,closeness
0,4


## tg_resource_allocation

Resource Allocation is used to compute the closeness of nodes based on their shared neighbors. (https://docs.tigergraph.com/graph-ml/current/link-prediction/resource-allocation)

## Input Parameters

* VERTEX v_source: The first vertex to compare {"id": "vertex_id", "type": "vertex_type"}
* VERTEX v_target: The second vertex to compare with the first {"id": "vertex_id", "type": "vertex_type"}
* SET<STRING> e_type_set: Edge types to traverse 
* BOOL print_results: if True, print result (True by default)

In [25]:
params = {
    "v_source": {"id": "Alex", "type": "Person"},
    "v_target": {"id": "Bob", "type": "Person"},
    "e_type_set": ["Coworker"],
    "print_results": True
}

In [26]:
start_time = time.time()

algo_memory = %memit -r 1 -o feat.runAlgorithm("tg_resource_allocation", params=params)

algo_memory = str(algo_memory)

start = algo_memory.find(": ") + 1
end = algo_memory.find("M")

algo_memory = algo_memory[start:end].strip()

execution_time = time.time() - start_time

cpu_usage = psutil.cpu_percent(4)

print('The CPU usage is: ', cpu_usage)

# print('RAM memory % used:', psutil.virtual_memory()[2])

host_memory = psutil.virtual_memory()[3]/1000000000

print('RAM Used (GB):', host_memory)

print ('tg_resource_allocation executed successfully')

print ('execution time: ' + str(execution_time) + ' seconds\n')

algo_id = "tg_resource_allocation_" + config["job_id"]

nb_id = "algos_topologicalLinkPrediction.ipynb_" + config["job_id"]

data = [algo_id, "false" ,cpu_usage, algo_memory, execution_time, host_memory, "3.8", "no error", nb_id]

with open(algo_performance_out, mode='a+', encoding='utf-8') as f:
    writer = csv.writer(f) 
    writer.writerow(data)

Installing and optimizing the queries, it might take a minute...
Queries installed successfully
peak memory: 126.84 MiB, increment: 0.12 MiB
The CPU usage is:  23.8
RAM Used (GB): 11.362390016
tg_resource_allocation executed successfully
execution time: 36.40755581855774 seconds



In [27]:
results = feat.runAlgorithm("tg_resource_allocation", params=params)

## Results

Returns a closeness value for two input vertices.

In [28]:
df_resource_allocation = pd.json_normalize(results)

display(df_resource_allocation)

,@@sum_closeness
0,0.5


## tg_total_neighbors

The algorithm counts the total number of neighbors, or vertices connected by one hop, of two vertices. (https://docs.tigergraph.com/graph-ml/current/link-prediction/total-neighbors)

## Input Parameters

* VERTEX v_source: The first vertex to compare {"id": "vertex_id", "type": "vertex_type"}
* VERTEX v_target: The second vertex to compare with the first {"id": "vertex_id", "type": "vertex_type"}
* SET<STRING> e_type_set: Edge types to traverse 
* BOOL print_results: if True, print result (True by default)

In [29]:
params = {
    "v_source": {"id": "Alex", "type": "Person"},
    "v_target": {"id": "Bob", "type": "Person"},
    "e_type_set": ["Coworker"],
    "print_results": True
}

In [30]:
start_time = time.time()

algo_memory = %memit -r 1 -o feat.runAlgorithm("tg_total_neighbors", params=params)

algo_memory = str(algo_memory)

start = algo_memory.find(": ") + 1
end = algo_memory.find("M")

algo_memory = algo_memory[start:end].strip()

execution_time = time.time() - start_time

cpu_usage = psutil.cpu_percent(4)

print('The CPU usage is: ', cpu_usage)

# print('RAM memory % used:', psutil.virtual_memory()[2])

host_memory = psutil.virtual_memory()[3]/1000000000

print('RAM Used (GB):', host_memory)

print ('tg_total_neighbors executed successfully')

print ('execution time: ' + str(execution_time) + ' seconds\n')

algo_id = "tg_total_neighbors_" + config["job_id"]

nb_id = "algos_topologicalLinkPrediction.ipynb_" + config["job_id"]

data = [algo_id, "false" ,cpu_usage, algo_memory, execution_time, host_memory, "3.8", "no error", nb_id]

with open(algo_performance_out, mode='a+', encoding='utf-8') as f:
    writer = csv.writer(f) 
    writer.writerow(data)

Installing and optimizing the queries, it might take a minute...
Queries installed successfully
peak memory: 126.92 MiB, increment: 0.14 MiB
The CPU usage is:  22.5
RAM Used (GB): 11.359371264
tg_total_neighbors executed successfully
execution time: 33.12188982963562 seconds



In [31]:
results = feat.runAlgorithm("tg_total_neighbors", params=params)

## Results

The total number of neighbors of two vertices as a closeness value.

In [32]:
df_total_neighbors = pd.json_normalize(results)

display(df_total_neighbors)

,closeness
0,3
